<a id="top"></a>
# AntakIA tutorial
***
### Using AntakIA with no GUI!

AntakIA helps you understand and explain your _black-box_ machine-learning models, by identifying the most relevant way of segregating your dataset and the best surrogate models to apply on these freshly created regions. In this notebook, we will show you how to use the automatic dyadic-clustering algorithm of AntakIA.

> This notebook is a tutorial on how to use AntakIA without the GUI. If you want to use the GUI, please refer to the [AntakIA GUI tutorial](antakia_gui.ipynb).
> 
> For more information about AntakIA, please refer to the [AntakIA documentation](https://ai-vidence.github.io/antakia/) or go to [AI-vidence's website](https://ai-vidence.com/).

__In this notebook, you will learn how to:__
- Create a dataset object from a CSV file
- Instanciate an AntakIA object
- Manually define regions and apply sub-models
- Run the automatic dyadic-clustering algorithm
- Visualize the results

## Context :

__Let's pretend that we are a real estate agent and that we want to predict the price of a house based on its characteristics.__ We have a dataset of more than 20000 blocks of houses, each block being described by 8 features (e.g. medium income of the owners, number of rooms, etc.). We also have the price of each block of houses. We already trained a machine-learning model (in our case, a simple XGBoost) that will predict the price of a house based on its characteristics. This is very helpful to estimate the price of a house that we want to sell !

__The main issue is the following :__ we want to explain to our customers why their house is worth a certain price. We can't just show them the machine-learning model, because it is a _black-box_ model. We need to find a way to explain the price of a house based on its characteristics. This is where AntakIA comes in handy !

We start by importing the necessary libraries.

In [1]:
import pandas as pd 

Then, our dataset. Ours is [this one](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html); it can be found in the `data` folder of this repository.

In [2]:
df = pd.read_csv('../data/california_housing/california_housing.csv').drop(['Unnamed: 0'], axis=1)

After cleaning a bit our data, we want to specifically focus on __San Francisco__ and its surroundings.

In [3]:
# Remove outliers:
df = df.loc[df['Population']<10000] 
df = df.loc[df['AveOccup']<6]
df = df.loc[df['AveBedrms']<1.5]
df = df.loc[df['HouseAge']<50]

# Only San Francisco :
df = df.loc[(df['Latitude']<38.07)&(df['Latitude']>37.2)]
df = df.loc[(df['Longitude']>-122.5)&(df['Longitude']<-121.75)]

Note that we already computed some explanatory values (in our case, SHAP values) and saved them in the CSV file. This is not necessary, as AntakIA can do it, but it will save us some computation time!

In [4]:
X = df.iloc[:,0:8] # the dataset
Y = df.iloc[:,9] # the target variable
SHAP = df.iloc[:,[10,11,12,13,14,15,16,17]] # the SHAP values

We also have a trained XGBoost model that we will use to predict the price of a house.

In [5]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state = 9)
model.fit(X, Y)

GradientBoostingRegressor(random_state=9)

__Let's now import `antakia`!__

In [6]:
import antakia

## 1. Creating the dataset object

We first use the [`Dataset`](https://ai-vidence.github.io/antakia/documentation/dataset/) class to create a dataset object. This object will be used to store the data and the machine-learning model.

In [7]:
dataset = antakia.Dataset(X, model = model, y=Y)
print(f'Size of the data we want to explore: {len(dataset)} lines')

Size of the data we want to explore: 2799 lines


## 2. Creating the AntakIA object
We then use the [`AntakIA`](https://ai-vidence.github.io/antakia/documentation/antakia/) class to create an AntakIA object. This is the main object of the package!
This is where we import our explanatory values (in our case, SHAP values).

In [8]:
atk = antakia.AntakIA(dataset, import_explanation = SHAP)

## 3. Launching the GUI!
We can now launch the GUI! We will use it to manually define regions, explore our data and model, or run the automatic dyadic-clustering algorithm.

Before using the GUI, we recommend you to do the following things:
- Take a look at its documentation [here](https://ai-vidence.github.io/antakia/documentation/gui/). `GUI` is a specific AntakIA object!
- Read the [User guide](https://ai-vidence.github.io/antakia/usage/) section of the documentation.
- Watch the video tutorials on [AI-vidence's website](https://ai-vidence.com/) or on [YouTube](https://www.youtube.com/@AI-vidence).

In [ ]:
atk.startGUI()

__You may retrieve your results by using the following commands:__

In [10]:
print(atk.gui.getSelection()) # the selection is an attribute of the GUI. It is a Potato object!

Potato:
 ------------------
       State: unknown 
       Number of points: 0 
       Percentage of the dataset: 0.0% 
       Sub-model: NoneType


In [11]:
atk.getRegions() # get the regions created using the GUI. A region is a list of antakia Potatoes.

[]

In [12]:
atk.getSaves() # get the saves created using the GUI. A save is a list of regions.

[]

In [13]:
atk.getExplanations() # get the explanation values (Imported, SHAP, LIME), so as to save them locally and use them later. Their computation can be long!

{'Imported':       MedInc_shap  HouseAge_shap  AveRooms_shap  AveBedrms_shap  \
 0        1.908911       0.167049       0.108704       -0.015404   
 1        1.625067      -0.054438       0.032429       -0.091859   
 2       -0.858459       0.088403       0.038161        0.024845   
 3       -0.825337       0.037305      -0.049962        0.005793   
 4       -0.772103       0.044774      -0.046207       -0.009898   
 ...           ...            ...            ...             ...   
 2794    -0.483621      -0.081927      -0.027561       -0.016950   
 2795     0.088662       0.035506       0.033724       -0.011320   
 2796     0.001963      -0.042058      -0.020210       -0.014419   
 2797     0.283789      -0.153535      -0.002481       -0.030824   
 2798    -0.350758       0.032192      -0.046780        0.008140   
 
       Population_shap  AveOccup_shap  Latitude_shap  Longitude_shap  
 0           -0.022898       0.063015      -0.271882        0.341710  
 1            0.044110      

That's it ! You now know how to use AntakIA with GUI. If you don't want to use the GUI, please refer to the [AntakIA with no GUI tutorial](antakia_no_gui.ipynb).
***

## List if usefull links

- [AntakIA documentation](https://ai-vidence.github.io/antakia/) - The official documentation of AntakIA
- [AntakIA GitHub repository](https://github.com/AI-vidence/antakia/tree/main) - The GitHub repository of AntakIA. Do not forget to __star__ it if you like it!
- [AntakIA video tutorials](https://www.youtube.com/@AI-vidence) - The YouTube channel of AI-vidence, with video tutorials on AntakIA!
- [AI-vidence's website](https://ai-vidence.com/) - The website of AI-vidence, the company behind AntakIA

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/AI-vidence/antakia/main/docs/img/Logo-AI-vidence.png" alt="AI-vidence" width="200px"/> 

 ***